In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

C:\Users\hp\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
df = pd.read_csv("Churn_Modelling.csv")

In [3]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [5]:
df.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [6]:
x_columns = df.columns.tolist()[3:13]
y_columns = df.columns.tolist()[-1:]
print(x_columns)
print(y_columns)

['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
['Exited']


In [7]:
x = df[x_columns].values
y = df[y_columns].values
print("x = ", x)
print("y = ", y)

x =  [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y =  [[1]
 [0]
 [1]
 ...
 [1]
 [1]
 [0]]


In [8]:
from sklearn.preprocessing import LabelEncoder
x_column_transformer = LabelEncoder()
x[:, 1] = x_column_transformer.fit_transform(x[:, 1])

In [9]:
x[:, 2] = x_column_transformer.fit_transform(x[:, 2])

In [10]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    [
        ('Categorizer', ColumnTransformer(
            [
                ("Gender Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [2]),
                ("Geography Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [1])
            ], 
            remainder = 'passthrough', n_jobs = 1)),
        ('Normalizer', StandardScaler())
    ]
)

In [11]:
x = pipeline.fit_transform(x)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [13]:
import tensorflow as tf

In [14]:
# pip install keras

In [15]:
# pip install tensorflow

In [16]:
# pip install --upgrade numpy

In [17]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [18]:
classifier = Sequential()

In [19]:
classifier.add(Dense(6, activation = 'relu', input_shape = (X_train.shape[1], )))
classifier.add(Dropout(rate = 0.1))

In [20]:
classifier.add(Dense(6, activation = 'relu'))
classifier.add(Dropout(rate = 0.1))

In [21]:
classifier.add(Dense(1, activation = 'sigmoid'))

In [22]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dropout (Dropout)           (None, 6)                 0         
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dropout_1 (Dropout)         (None, 6)                 0         
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 121 (484.00 Byte)
Trainable params: 121 (484.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [25]:
history = classifier.fit(X_train, y_train, batch_size = 32, epochs = 100, validation_split = 0.1, verbose = 2)

Epoch 1/100
225/225 - 1s - loss: 0.3983 - accuracy: 0.8415 - val_loss: 0.3568 - val_accuracy: 0.8612 - 570ms/epoch - 3ms/step
Epoch 2/100
225/225 - 1s - loss: 0.3941 - accuracy: 0.8440 - val_loss: 0.3554 - val_accuracy: 0.8612 - 571ms/epoch - 3ms/step
Epoch 3/100
225/225 - 1s - loss: 0.3942 - accuracy: 0.8453 - val_loss: 0.3537 - val_accuracy: 0.8575 - 580ms/epoch - 3ms/step
Epoch 4/100
225/225 - 1s - loss: 0.3923 - accuracy: 0.8492 - val_loss: 0.3526 - val_accuracy: 0.8662 - 592ms/epoch - 3ms/step
Epoch 5/100
225/225 - 1s - loss: 0.3920 - accuracy: 0.8458 - val_loss: 0.3500 - val_accuracy: 0.8675 - 603ms/epoch - 3ms/step
Epoch 6/100
225/225 - 1s - loss: 0.3929 - accuracy: 0.8440 - val_loss: 0.3498 - val_accuracy: 0.8662 - 684ms/epoch - 3ms/step
Epoch 7/100
225/225 - 1s - loss: 0.3904 - accuracy: 0.8456 - val_loss: 0.3487 - val_accuracy: 0.8650 - 607ms/epoch - 3ms/step
Epoch 8/100
225/225 - 1s - loss: 0.3879 - accuracy: 0.8479 - val_loss: 0.3488 - val_accuracy: 0.8625 - 598ms/epoch - 3

In [26]:
y_pred = classifier.predict(X_test)
print(y_pred[:5])

63/63 [==============================] - 0s 2ms/step
[[0.23524319]
 [0.28624055]
 [0.14680599]
 [0.06220995]
 [0.15009077]]


In [27]:
y_pred = (y_pred > 0.5).astype(int)
print(y_pred[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [28]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1526   69]
 [ 196  209]]


In [29]:
print(((cm[0][0] + cm[1][1])* 100) / len(y_test), '% of data was classified correctly')

86.75 % of data was classified correctly
